In [31]:
from os import environ
from os.path import join

import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import nltk

In [32]:
api_url = "http://api.genius.com"
access_token = "Eaoa-I_DHC92LYuNAPbdh9o0-D-aXrl7T3aGUOOw-e6OrwymEoULjMgcVHC-1ZXS"
headers = {
    "Authorization": f"Bearer {access_token}"
}


In [33]:
# Get each artist url and ID with metadata in <head>
# meta[name="newrelic-resource-path"] on console
# "%22" means the double quotes character "
artists = {
    "%22Nekfeu%22": ("13063", "", {}, pd.DataFrame()),
    "%22Ninho%22": ("71286", "", {}, pd.DataFrame()),
    "%22Damso%22": ("45855", "", {}, pd.DataFrame()),
    "%22OrelSan%22": ("1286", "", {}, pd.DataFrame()),
    "%22SCH%22": ("276476", "", {}, pd.DataFrame()),
    "%22Oxmo Puccino%22": ("1947", "", {}, pd.DataFrame()),
    "%22Booba%22": ("1282", "", {}, pd.DataFrame()),
    "%22Vald%22": ("19217", "", {}, pd.DataFrame()),
    "%22Niska%22": ("234413", "", {}, pd.DataFrame()),
    "%22Akhenaton%22": ("1961", "", {}, pd.DataFrame())
}


In [88]:
# Get song name
def get_song_name(all_text: str) -> str:
    dash_index = all_text.find('–')
    pipe_index = all_text.find('|')
    song_name = all_text[dash_index + 1:pipe_index].strip()
    song_name = song_name.replace("'", "’")

    return song_name

def check_featuring(all_text: str) -> int:
    featuring = 0
    check = all_text.find("Featuring")
    if check != -1:
        featuring = 1

    return featuring

# Remove additional info before lyrics
def delete_pre_lyrics(all_text: str, song_name: str) -> str:
    # Remove the song name at the beginning
    song_name_index = all_text.find(song_name)
    tmp = all_text[song_name_index + len(song_name) + 1:]
    
    # Remove everything before the second song name statement before the lyrics
    song_name_index = tmp.find(song_name)
    if song_name_index != -1:
        tmp = tmp[song_name_index + len(song_name) + 1:]
    
    # Remove "[Paroles de...] statement if present"
    song_name = song_name.replace("’", "'")
    song_name = song_name.replace(" Lyrics", "")
    parole_statement = f"[Paroles de \"{song_name}\""
    parole_index = tmp.find(parole_statement)
    if parole_index != -1:
        end_index = tmp.find("|", parole_index)
        tmp = tmp[end_index + 1:]

    return tmp

# Remove additional info after lyrics
def delete_post_lyrics(lyrics_with_post: str) -> str:
    end_index = lyrics_with_post.find("Embed")
    lyrics = lyrics_with_post[:end_index]
    lyrics = re.sub(r'\|\d+\|$', '', lyrics)

    return lyrics

# Remove all non-artist lyrics
def remove_other_artist_interventions(lyrics, author):
    pattern = r'\[([^\]]+)\](.*?(?=\[|$))'
    
    def repl(match):
        artist = match.group(1)
        if artist == author or f"{author}]|" in match.group(0):
            return match.group()
        else:
            return ''
    
    new_lyrics = re.sub(pattern, repl, lyrics)
    return new_lyrics

# Remove all non-artist lyrics
def get_artist_lyrics(lyrics: str, artist: str, featuring: int) -> str:
    # Remove "You might also like" statements in lyrics and "%22" from artist name
    lyrics = lyrics.replace("|You might also like", "")
    artist = artist.replace("%22", "")
    
    if featuring:
        # Remove all other artists interventions
        lyrics = remove_other_artist_interventions(lyrics, artist)
        # Remove all balise []
        while lyrics.find('[') != -1:
            start = lyrics.find("[")
            end = lyrics.find("]", start)
            lyrics = lyrics[:start] + lyrics[end + 2:]
    else:
        # Remove all balise []
        while lyrics.find('[') != -1:
            start = lyrics.find("[")
            end = lyrics.find("]", start)
            lyrics = lyrics[:start] + lyrics[end + 2:]

    lyrics = lyrics.replace("|", " ")

    return lyrics

In [89]:
def get_lyrics(all_text: str, song_name: str, artist: str) -> str:
    # Check if multiple artists are in the song
    featuring = check_featuring(all_text)

    # Remove additional info before lyrics
    lyrics_with_post = delete_pre_lyrics(all_text, song_name)

    # Remove additional info after lyrics
    lyrics = delete_post_lyrics(lyrics_with_post)

    # Remove all non-artist lyrics
    cleaned_lyrics = get_artist_lyrics(lyrics, artist, featuring)

    return cleaned_lyrics

In [90]:
for artist, value in artists.items():
    playlist = {}
    # Form the url of the artist
    url = join(api_url, f"search?q={artist}")
    print(artist, ": ", url)

    # Request to get max musics of the artist
    r = requests.request("GET", url, headers=headers)

    if r.status_code == 200:
        # Take the songs from the response
        data = r.json()
        songs = data["response"]["hits"]
        
        if songs:
            # Extract URLs for each song
            songs_urls = [song["result"]["url"] for song in songs]

            for song_url in songs_urls:
                # Request to get the song page content
                song_response = requests.request("GET", song_url, headers=headers)
                if song_response.status_code == 200:
                    # Get the html of the page response
                    song_soup = BeautifulSoup(song_response.content, "html.parser")

                    # Retrieve all the text and concat with '|'
                    all_text = song_soup.get_text("|", strip=True)
                    song_name = get_song_name(all_text)

                    # Retrieve the lyrics of the specified artist in the specified song's text
                    lyrics = get_lyrics(all_text, song_name, artist)
                else:
                    print(f"Request for {song_url} failed !")

                # Add the song as key in the dictionary and the corresponding lyrics as value
                playlist[song_name] = lyrics

            # Add URL and playlist in the "artist" key of the "artists" dictionary
            artists[artist] = (value[0], url, playlist, value[3])
    else:
        print(f"Request for {artist} failed !")

%22Nekfeu%22 :  http://api.genius.com/search?q=%22Nekfeu%22
Nekfeu
Solo
[Couplet 1]|Est-ce que tu t'es d'jà fait rabaisser par celle que t'aimais secrètement ?|Gentille en privé mais, d'vant les gens, cruelle et légère|Est-ce que t'as d'jà tapé quelqu'un juste pour qu'on te respecte ?|Sans excuse, est-ce que t'as d'jà regretté au point d'ber-ger ?|Est-ce que tu t'es d'jà dit : "Faut qu'j'me secoue, j'ai plus 16 ans" ?|N'être plus qu'une âme perdue, errer et rôder des heures|Voir un mec s'faire racketter, appeler au s'cours et pisser l'sang|Dans le même wagon du RER où t'étais seul|Est-ce que t'as détourné l'regard ? Dégoûté face à ta propre lâcheté|Est-ce qu'après t'as fait des trucs de malade juste pour t'racheter ?|Est-ce pour ça qu't'as été violent quand la prof t'a jeté ?|Est-ce que tu t'es identifié au taf de Kourtrajmé ?|Est-ce que tu gardais, à l'époque|Des secrets d'famille lourds dont tu peux même pas parler à tes potes ?|Même pas deux balles pour un café, rêvais-tu d'sauter l

KeyboardInterrupt: 

In [ ]:
# for artist, value in artists.items():
    # df = count_persons(value[2])
    # print(df)
    # artists[artist] = (value[0], value[1], value[2], df)   # Add the DataFrame with person count in each lyrics of the current artist